In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [3]:
import pandas as pd

# Dosyaları oku (doğrudan bulunduğu dizinden)
temperature = pd.read_csv('temperature.csv')
humidity = pd.read_csv('humidity.csv')
pressure = pd.read_csv('pressure.csv')

# datetime sütununu indeks yap
temperature['datetime'] = pd.to_datetime(temperature['datetime'])
humidity['datetime'] = pd.to_datetime(humidity['datetime'])
pressure['datetime'] = pd.to_datetime(pressure['datetime'])

temperature.set_index('datetime', inplace=True)
humidity.set_index('datetime', inplace=True)
pressure.set_index('datetime', inplace=True)

# Ortak tarih aralığında verileri birleştir
merged = pd.concat([temperature, humidity, pressure], axis=1, keys=['temp', 'hum', 'pres'])


In [5]:
# Şehir listesi otomatik olarak
city_list = temperature.columns.tolist()

print("Kullanılabilir şehirler:")
print(city_list)

# Kullanıcıdan şehir seçimi al
city = input("Tahmin yapılacak şehir adını girin: ")

if city not in city_list:
    raise ValueError(f"{city} veride bulunamadı. Lütfen geçerli bir şehir girin.")


Kullanılabilir şehirler:
['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque', 'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal', 'Boston', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa', 'Nahariyya', 'Jerusalem']
Tahmin yapılacak şehir adını girin: Portland


In [6]:
# Şehir verisi (temp, hum, pres)
df = pd.DataFrame({
    'temperature': merged['temp'][city],
    'humidity': merged['hum'][city],
    'pressure': merged['pres'][city],
}).dropna()

# Normalize et
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

# Zaman penceresi oluştur (72 saatlik giriş → 1 hedef değer)
def create_sequences(data, window=72):
    X, y = [], []
    for i in range(len(data) - window - 72):  # 72 saat sonrası tahmin için
        X.append(data[i:i+window])
        y.append(data[i+window+72][0])  # sadece sıcaklık tahmini
    return np.array(X), np.array(y)

X, y = create_sequences(scaled)


In [7]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(1)  # sıcaklık çıkışı
])

model.compile(optimizer='adam', loss='mse')
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 72, 64)         │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,545 (205.25 KB)

 Trainable params: 52,545 (205.25 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(X, y, epochs=2, batch_size=32, validation_split=0.2)


Epoch 1/2
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 98s 83ms/step - loss: 0.0118 - val_loss: 0.0064
Epoch 2/2
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 91s 82ms/step - loss: 0.0075 - val_loss: 0.0063


In [9]:
# En son 72 saatlik veriyle 3 gün sonrası tahmini yap
last_seq = scaled[-72:]
last_seq = np.expand_dims(last_seq, axis=0)
pred_scaled = model.predict(last_seq)[0][0]

# Sıcaklık değeri olarak geri dönüştür
temp_min = scaler.data_min_[0]
temp_max = scaler.data_max_[0]
pred_temp = pred_scaled * (temp_max - temp_min) + temp_min

print(f"\n🌡️ {city} için 3 gün sonrası tahmin edilen sıcaklık: {pred_temp:.2f} Kelvin")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step

🌡️ Portland için 3 gün sonrası tahmin edilen sıcaklık: 281.13 Kelvin
